# Submission of Calculation with Subsequent Results Query

In this demo, we will submit a calculation, check the status of the job and query the results after it is finished.

## Initialization of Workspace 

Specify the Azure quantum workspace and the 'connection string' which allows us to connect to the workspace

In [ ]:
from azure.quantum import Workspace
from azure.quantum.job import JobFailedWithResultsError

# insert connection string from Azure Portal Workspace Access Keys
connection_string = "" 
workspace = Workspace.from_connection_string(connection_string)

In [ ]:
# To submit Accelerated DFT jobs, we will be using the microsoft.dft target in the workspace.
print("Verifying access to Accelerated DFT target.")
target = workspace.get_targets("microsoft.dft")
print("Verification complete.")

## Define Input and Submit Accelerated DFT job

In [ ]:
# First, let's define the molecular structure, loaded from an xyz file.
from pathlib import Path
GeomFile = "molecules/sih4.xyz"

In [ ]:
# Secondly, let's give a name for the job.
job_name = 'sih4_go_ts'

Now we submit the calculations to MADFT service.

In [ ]:
# Next, we create a dictionary variable to specify the parameters for the DFT calculation.

dft_input_params = {
  "tasks": [
    {
      "taskType": "go", 
      "basisSet": { "name": 'def2-svpd'},
      "xcFunctional": { "name": "b3lyp", "gridLevel": 4 },
      "molecule": { "charge": 0, "multiplicity": 1 },
      "scf": { "method": "rks", "maxSteps": 100, "convergeThreshold": 1e-8 },
      # We can optionally adjust the convergence criteria for the energy, gradient and displacement:
      # (see https://geometric.readthedocs.io/en/latest/options.html)  
      "geometryOptimization": {"transition": True, "convergence_energy": 1e-5, "convergence_grms": 1.7e-3, "convergence_gmax": 2.5e-3, "convergence_drms": 6.7e-3, "convergence_dmax": 1.0e-2 }
    }
  ]
}

# We are now ready to submit the Job using the target.submit call. It takes three parameters-
# 1. The input molecule in xyz format.
# 2. The DFT parameters that we declared above.
# 3. A friendly name to help identify the job in the Azure Portal later.

print("Submitting DFT job.")

job = target.submit(
    input_data=Path(GeomFile).read_text(),
    input_params = dft_input_params,
    name= job_name)
    
print("\nDFT job has been submitted.")
print(f"\nJob name: {job_name}")


Show the status of the job. If the job has finished, read the results of the job

In [ ]:
job.refresh()
print(f'Job: "{job_name}" is {job.details.status}')
if job.details.status == 'Succeeded':
    qcschema = job.get_results()["results"][0]

## Output to QCSchema json file

In [ ]:
import json
qcschema_json = job_name + "_output.json"
with open(qcschema_json, "w") as fp:
    json.dump(qcschema, fp)

## Results

The results of the calculation are stored in the QCSchema format dict.

We can print the energy of the optimized structure:

In [ ]:
print("Total Energy of optimized geometry (Hartree): ", qcschema["energies"][-1])

In [ ]:
qcschema['provenance']['total_time_seconds']

We can print the coordinates of the optimized geometry. Note that QCSchema output uses Bohr.

In [ ]:
import numpy as np
coords = np.array(qcschema["final_molecule"]["geometry"] )
print( np.reshape(coords, (-1,3)) )

Let us save the optimized geometry as an xyz file (and convert to Angstrom) so that we can easily view it and read in for subsequent calculations.

In [ ]:
from tools.libqcschema import *
# convert to Angstrom
opt_xyz = load_qcschema_final_molecule(qcschema, to_Angstrom = True, xyz=True)
print(opt_xyz)
geom_opt_file = "phenol_optimized_geometry.xyz"
with open(geom_opt_file,'w') as f:
    f.write(opt_xyz)


Let us view the optimized geometry...

In [ ]:
# For visualization of molecules and orbitals:
import py3Dmol

data = None
with open(geom_opt_file, "r") as infile:
    data = infile.read()

view = py3Dmol.view()
view.addModel(data)
view.setStyle({"stick": {}})
# if you prefer ball and stick molecules:
#view.setStyle({"stick":{},"sphere": {"scale":0.25}})
view.show()